<div style="text-align: center;">
  
# 🧁 Limpeza e tratamento de dados de CGM (monitoramento contínuo de glicemia)

</div>

## 1. Carregando as bibliotecas

In [ ]:
#%pip install missingno

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## 2. Extraindo e visualizando o dataframe; padronizando os nomes das variáveis; certificando os tipos de dados

In [ ]:
# Dataset extraído pelo RStudio através do repositório 
# https://rdrr.io/github/personalscience/psi-shiny-cgm/man/sample_libreview_df.html
df_extract = pd.read_csv('./sample_libreview_df_raw.csv')
df_raw = df_extract.copy()

In [ ]:
'''
No atributo columns (do tipo índice) do objeto df_raw, nosso dataframe, iremos formatar
todos os seus elementos, isto é, os nomes das colunas. Para isso, utilizamos o acessor .str, 
que nos permite aplicar métodos de strings para todos os elementos do índice/vetor.
Os métodos utilizados são: 
- lower(): torna todos os caracteres minúsculos;
- strip(): remove espaços em branco das extremidades da string;
- replace(,): substituímos o elemento da esquerda, sempre que ele aparecer, pelo da 
direita. No nosso caso, trocamos um espaço em branco por nenhum espaço, e ":" por nenhum espaço.
'''

df_raw.columns = df_raw.columns.str.lower().str.strip().str.replace(' ', '').str.replace(':', '')
df_raw

In [ ]:
# convertemoscoluna 'time' para o tipo datetime
df_raw['time'] = pd.to_datetime(df_raw['time'], errors = 'coerce') 
# a coluna 'value' se torna numérica
df_raw['value'] = pd.to_numeric(df_raw['value'], errors = 'coerce') 

# OBS: errors = 'coerce' faz com que elementos que não estejam num formato válido para
# serem convertidos para datetime retornem NaT, e os que não possam ser convertidos para
# int ou float (numérico) retornem NaN.

## 3. Atribuindo valores reais (esquecidos) a 'value', seguindo a hierarquia entre 'strip', 'scan' e 'hist'

In [ ]:
def value_herda_exclui(df, col_time='time', col_alvo='value', col1='strip', col2='scan', col3='hist'):
    """
    Preenche os NaNs de `col_alvo` com prioridade:
    1. Usa `col1` se disponível.
    2. Caso contrário, usa `col2`.
    3. Caso contrário, usa `col3`.
    """
    
    df = df.copy()
    
    # 0 = não foi preenchido por outra coluna, 1 foi por col1, 2 por col2 e 3 por col3.
    df['preenchido_por'] = 0

    # vetores (Series), onde: o primeiro tem valores True nos índices em que col_alvo é NaN; 
    # o segundo tem valores True onde col1 não é NaN
    cond1 = df[col_alvo].isna() & df[col1].notna() 
    # faço a substituição nessas posições
    df.loc[cond1, col_alvo] = df.loc[cond1, col1]
    df.loc[cond1, 'preenchido_por'] = 1

    # Faço o mesmo com as linhas restantes de col_alvo que continuam NaN, mas agora
    # comparando com os valores de col2 e col3
    cond2 = df[col_alvo].isna() & df[col2].notna()
    df.loc[cond2, col_alvo] = df.loc[cond2, col2]
    df.loc[cond2, 'preenchido_por'] = 2

    cond3 = df[col_alvo].isna() & df[col3].notna()
    df.loc[cond3, col_alvo] = df.loc[cond3, col3]
    df.loc[cond3, 'preenchido_por'] = 3

    # Agora, vamos ordenar por prioridade e eliminar duplicatas no tempo
    df[col_time] = pd.to_datetime(df[col_time], errors = 'coerce') 
    df = df.sort_values(by=[col_time, 'preenchido_por'])  # menor preenchido_por tem maior prioridade
    df = df.drop_duplicates(subset=col_time, keep='first').reset_index(drop=True)

    # Removo a coluna que indica de qual medição o value foi advindo
    df = df.drop(columns=['preenchido_por'])

    return df

df_raw = value_herda_exclui(df_raw)
df_raw

## 4. Removendo as variáveis restantes e, agora, desnecessárias

In [ ]:
# Eliminando as colunas que não serão utilizadas
df_raw = df_raw.drop(columns=['unnamed0', 'strip', 'hist', 'scan', 'food', 'user_id'])

df_raw

## 5. Eliminando os valores nulos e absurdos

In [ ]:
# Contagem de quantos valores nulos (NaN ou NaT) temos em cada coluna de df_raw
# df_raw.isnull() retorna um dataframe com True e False em cada célula. True, se a célula tem
# valor NaN ou NaT, e False em caso contrário. O método sum(axis = 0) soma a quantidade de cada coluna.
# Como True corresponde ao valor 1 e False a 0, teremos assim a soma dos valores nulos.

print('Valores nulos antes: \n')
print(df_raw.isnull().sum(axis = 0))

In [ ]:
def elimina_nulos_ruidos(df, col_time='time', col_value='value', lim_inf=0, lim_sup=500, minutes_rel1=15, minutes_rel2=15, ruido_rel=50):
    '''Documentação'''

    df = df.copy()

    # Garantir os tipos de dados
    df_raw['time'] = pd.to_datetime(df_raw['time'], errors = 'coerce') 
    df_raw['value'] = pd.to_numeric(df_raw['value'], errors = 'coerce') 

    # Excluindo as linhas em que 'time' ou 'value' tem valor nulo
    df = df.dropna(subset=[col_time,col_value])

    # Excluindo as linhas em que 'value' tem um possível ruído absoluto
    df = df[(df[col_value] > lim_inf) & (df[col_value] < lim_sup)]

    # Excluindo as linhas em que 'value' tem um possível ruído relativo
    df['diff_prev'] = df[col_value].diff().abs() # diferença de valor para a linha anterior
    df['diff_next'] = df[col_value].diff(-1).abs() # diferença de valor para a linha posterior

    df['delta_prev'] = df[col_time] - df[col_time].shift(1) # diferença de tempo para a linha anterior
    df['delta_next'] = df[col_time].shift(-1) - df[col_time] # diferença de tempo para a linha posterior

    mask_delta_prev = df['delta_prev'] <= pd.Timedelta(minutes=minutes_rel1)
    mask_delta_next = df['delta_next'] <= pd.Timedelta(minutes=minutes_rel2)

    mask_ruido_duplo = (df['diff_prev'] > ruido_rel) & mask_delta_prev & \
                    (df['diff_next'] > ruido_rel) & mask_delta_next

    df = df.loc[~mask_ruido_duplo]
    df.drop(columns=['diff_prev', 'diff_next', 'delta_prev', 'delta_next'], inplace=True)

    # Organizando o dataframe por 'time' crescente, e resetando os índices, já que
    # ao excluirmos linhas, a linha com o índice é excluída. Agora, temos índices ordenados e
    # sequenciados novamente de 0 ao fim do dataframe.
    df = df.sort_values(by=col_time).reset_index(drop=True) 

    return df

# Esse é o dataframe apenas com dados medidos
df_raw = elimina_nulos_ruidos(df_raw)
df_raw


## 6. Subdivisão de 'time' em 'date', 'hour', 'hour_minute', 'weekday' e 'hour_cont'

In [ ]:
def dividir_time(df, col_time='time'):
    """
    Adiciona colunas derivadas de tempo para análise estatística e manipulação temporal.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame com uma coluna datetime.
    col_time : str, default 'time'
        Nome da coluna que contém timestamps (deve ser do tipo datetime ou conversível).
    Retorna:
    --------
    pd.DataFrame
        DataFrame com as colunas adicionais:
            - 'date' : data (sem horário)
            - 'hour' : hora inteira
            - 'hour_minute' : string com hora e minuto no formato HH:MM
            - 'weekday' : nome do dia da semana
            - 'hour_cont' : hora com fração decimal (ex: 14.25 para 14:15)
    """
    df = df.copy()

    df[col_time] = pd.to_datetime(df[col_time])

    df['date'] = pd.to_datetime(df[col_time].dt.date)  # <- corrigido aqui
    primeiro_dia = df['date'].min()
    df['date_cont'] = (df['date'] - primeiro_dia).dt.days  # <- agora funciona

    df['hour'] = df[col_time].dt.hour
    df['hour_minute'] = df[col_time].dt.strftime('%H:%M')
    df['weekday'] = df[col_time].dt.day_name()
    df['hour_cont'] = (df[col_time].dt.hour + df[col_time].dt.minute / 60).round(2)

    return df

df_raw = dividir_time(df_raw)
df_raw

In [ ]:
# Agora criamos a coluna 'faixa' diretamente, sem gerar warning
colunas_ordenadas = ['time', 'date', 'date_cont', 'hour_minute', 'value', 'weekday', 'hour', 'hour_cont']
df_raw = df_raw[colunas_ordenadas]


## 9. Organização final da tabela

In [ ]:
# Organização final da tabela de valores reais
def faixa_horaria(hora):
    if 0 <= hora <= 6:
        return 'Madrugada'
    elif 7 <= hora <= 12:
        return 'Manhã'
    elif 13 <= hora <= 17:
        return 'Tarde'
    elif 18 <= hora <= 21:
        return 'Jantar'
    else:
        return 'Fim da noite'

df_raw.loc[:, 'faixa'] = df_raw['hour'].apply(faixa_horaria)
df_raw

<div style="text-align: center;">
  
# 🔍 Análise Exploratória de Dados

</div>

## 1. Informações da tabela df_raw

In [ ]:
print('Quantidade de linhas e colunas:\n', df_raw.shape, '\n\n---\n')
print('Colunas presentes:\n', df_raw.columns, '\n---\n')
print('Tipo de dados:\n', df_raw.dtypes, '\n\n---\n')
print('Quantidade de dados únicos:\n', df_raw.nunique())

## 2. Estatísticas gerais da variável 'value' e sua distribuição de frequências absolutas

In [ ]:
df_raw['value'].describe().round(1)

In [ ]:
print('Quantidade de aparições dos valores medidos para glucose no fluido intersticial (mg/dL)')
sns.histplot(df_raw['value'], kde=True)
plt.ylabel('Número de aparições')
plt.xlabel('Glicose (mg/dL)')

## 3. Estatísticas diárias de 'value'

In [ ]:
print('Quantidade de medições por dia da semana')
df_raw['weekday'].value_counts().plot.bar()
plt.ylabel('Quantidade de medições')
plt.xlabel('Dia da semana')

In [ ]:
print('Número de medições por data')
df_raw.groupby('date').size().plot(kind='bar', figsize=(15, 4), title='Quantidade de medições por data')
plt.ylabel('Quantidade de aferições')
plt.xlabel('Data')
plt.show()

Abaixo, temos a média por dia, mas com um porém: mesmo considerando dias com mais de 64 medições, ainda podem haver dias com medições irregulares, que não contemplem todos os ciclos do dia!

In [ ]:
print('Média das medições nos dias em que houve aferição')
contagem_por_dia = df_raw.groupby('date_cont')['value'].count()
dias_validos = contagem_por_dia[contagem_por_dia > 64].index
media_filtrada = df_raw[df_raw['date_cont'].isin(dias_validos)].groupby('date_cont')['value'].mean()

# Plota a média diária apenas para os dias com mais de 64 medições
media_filtrada.plot(marker='o', linestyle='-')

x = media_filtrada.index          # números do dia: 0, 1, 3, 10, etc.
y = media_filtrada.values         # médias de glicose

# Plotando com espaçamento proporcional
plt.plot(x, y, marker='o', linestyle='-')

# Rótulos dos eixos
plt.xlabel('Dia (contagem desde o início)')
plt.ylabel('Média de Glicose (mg/dL)')
plt.title(f'Média diária de glicose (apenas dias com mais de 64 medições)')

# Ativar grade
plt.grid(True)

plt.show()

In [ ]:
print('Quantidade de medições por hora')
contagem_horas_ord = df_raw['hour'].value_counts().sort_index()
contagem_horas_ord.plot.bar()
plt.ylabel('Quantidade de medições')
plt.xlabel('Hora do dia')

## 1. Há relação entre o valor da glicemia (value) e o horário do dia (time)? Em quais horas do dia há maiores picos de glicemia? Em quais horas do dia há menores valores de glicemia? 


In [ ]:
# Valores de média, mínimo e máximo valores de glicemia (mg/dL) por hora do dia
df_raw.groupby('hour')['value'].agg(['mean', 'min', 'max']).round(1)

Serve para analisar o ciclo como um todo!

In [ ]:
# Gráfico da média de glicemia por hora do dia 
media_por_hora = df_raw.groupby('hour')['value'].mean()
plt.figure(figsize=(10, 5))
sns.lineplot(x=media_por_hora.index, y=media_por_hora.values)
plt.title('Média de glicemia por hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.grid(True)
plt.xticks(range(24))
plt.show()

In [ ]:
media_por_faixa = df_raw.groupby(['date_cont', 'faixa'])['value'].mean().unstack()

media_por_faixa.plot(figsize=(12, 6), marker='o')

plt.xlabel('Dia (contagem)')
plt.ylabel('Glicose média (mg/dL)')
plt.title('Evolução diária da glicose por faixa horária')
plt.grid(True)
plt.legend(title='Faixa do dia', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 2. Nos dados analisados, qual a porcentagem do tempo o paciente está dentro, acima ou abaixo dos valores de referência ideais? 

##### Valores ideais:
##### TIR time in range: ≥ 70% do tempo entre 70-180 mg/dL
##### TBR time below range: < 4% do tempo abaixo de 70 mg/dL
##### TBR time very below range: < 1% abaixo de 54 mg/dL
##### TAR time above range: < 25% acima de 180 mg/dL

In [ ]:
total = len(df_raw)
tar = len(df_raw[df_raw['value'] > 180]) / total * 100
tir = len(df_raw[(df_raw['value'] >= 70) & (df_raw['value'] <= 180)]) / total * 100
tbr = len(df_raw[df_raw['value'] < 70]) / total * 100
tvbr = len(df_raw[df_raw['value'] < 54]) / total * 100

In [ ]:
tar_check = '✔️' if tar < 25 else '❌'
tir_check = '✔️' if tir >= 70 else '❌'
tbr_check = '✔️' if tbr < 4 else '❌'
tvbr_check = '✔️' if tvbr < 1 else '❌'

In [ ]:
print(f'| TAR time above range      | {tar:.1f}%  | < 25% acima de 180 mg/dL           | {tar_check} |')
print(f'| TIR time in range         | {tir:.1f}% |  ≥ 70% do tempo entre 70-180 mg/dL | {tir_check} |')
print(f'| TBR time below range      | {tbr:.1f}% | < 4% do tempo abaixo de 70 mg/dL   | {tbr_check} |')
print(f'| TBR time very below range | {tvbr:.1f}%  | < 1% abaixo de 54 mg/dL            | {tvbr_check} |')

# 3. Dentro do período de medição, houve algum dia em que a média de glicemia destoou muito do normal? Em quais momentos houve episódios de hiper (>180mg/dL) e hipoglicemia? (<70mg/dL)

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

mapa_medicoes = df_raw[['value']]
mapa_medicoes = mapa_medicoes.T

faixas = [0, 70, 180, df_raw['value'].max() + 1]

# Cores suaves e harmônicas:
colors = ['#7DA6C1', '#A3C293', '#D46256']
cmap = ListedColormap(colors)
norm = BoundaryNorm(faixas, cmap.N)

# Plot
plt.figure(figsize=(20, 2))
sns.heatmap(mapa_medicoes, cmap=cmap, norm=norm, cbar_kws={'label': 'Glicose (mg/dL)'})
plt.yticks([], [])
plt.xlabel("Leituras")
plt.title("Mapa de calor com destaque para medições individuais acima de 180 e abaixo de 70")
plt.show()


# 4. Quão rápido a curva de glicemia tende a voltar para os valores de referência ideais? Há alguma variável que influencia o tempo de retorno?

# 5. Há diferença significativa da média de glicemia entre dias da semana? (dias úteis, fins de semana)

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

tabela = df_raw.pivot_table(index='hour', columns='weekday', values='value', aggfunc='mean')
max_val = df_raw['value'].max()

colors = ['#E99A8B', '#A3C293', '#D46256'] 
glucose_range = [0, 70, 140, max_val + 1]

cmap = ListedColormap(colors)
norm = BoundaryNorm(glucose_range, cmap.N)

plt.figure(figsize=(6, 6))
sns.heatmap(tabela, annot=True, fmt=".1f", cmap=cmap, norm=norm, cbar_kws={'label': 'Glicose (mg/dL)'})
plt.title("Glicose média por hora do dia e por dia da semana")
plt.ylabel("Hora do dia")
plt.xlabel("Dia da semana")
plt.show()

In [ ]:
media_por_dia = df_raw.groupby('weekday')['value'].mean().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
]).reset_index()

mapa_mediageral = pd.DataFrame([media_por_dia['value'].values], columns=media_por_dia['weekday'].values)

plt.figure(figsize=(8, 2))
sns.heatmap(mapa_mediageral, annot=True, fmt=".1f", cmap="crest", cbar_kws={'label': 'Glicose média (mg/dL)'})
plt.title("Média de glicose no fluido intersticial por dia da semana")
plt.yticks([], []) 
plt.xlabel("Dia da semana")
plt.show()


# 6. Há variação de glicemia significativa durante o período noturno e durante a manhã (4h-8h)? (dawn phenomenon)

In [ ]:
# Gráfico da média de glicemia por hora do dia 
media_por_hora = df_raw.groupby('hour')['value'].mean()
plt.figure(figsize=(10, 5))
sns.lineplot(x=media_por_hora.index, y=media_por_hora.values)
plt.title('Média de glicemia por hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.grid(True)
plt.xticks(range(24))
plt.show()

# 7. Qual a glicemia média geral do período analisado?

In [ ]:
media_global = df_raw['value'].mean()
media_check = '✔️' if media_global > 70 and media_global < 180 else '❌'
print('Média geral:', media_global, media_check)

# 8. Qual o desvio padrão ou coeficiente de variação (CV%) da glicemia?

# 9. Quais dias em que houve mais instabilidade (maior variação)?

# 10. Há consistência nos dados? É possível prever quais horas do dia haverá oscilações significativas?

# 11. Há alguma diferença entre a média da primeira e última metade da amostra? Houve alguma diferença no controle ao longo do mês ou permaneceu constante?

In [ ]:
# 12. Número e duração média dos episódios de hipoglicemia e hiperglicemia
# 13. Velocidade média de subida e queda da glicemia
# 14. Tempo em hipoglicemia noturna (00h–6h)
# 17. Predição simples com regressão ou modelo de baseline

# Predição com regressão linear

In [ ]:
#%pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
x = df_raw[['hour']]
y = df_raw['value']

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
modelo = LinearRegression()
modelo.fit(x_treino, y_treino)


In [ ]:
y_pred = modelo.predict(x_teste)
rmse = np.sqrt(mean_squared_error(y_teste, y_pred))

print("Coeficiente angular (slope):", modelo.coef_[0])
print("Intercepto:", modelo.intercept_)
print("R²:", r2_score(y_teste, y_pred))
print("RMSE:", rmse)

In [ ]:
plt.scatter(x_teste, y_teste, color='gray', label='Real')
plt.plot(x_teste, y_pred, color='blue', linewidth=2, label='Previsão')
plt.title('Previsão da glicemia com base na hora do dia')
plt.xlabel('Hora do dia')
plt.ylabel('Glicemia (mg/dL)')
plt.legend()
plt.grid(True)
plt.show()